In [ ]:
import re
import sys
import os
import requests
import time
import csv
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse
from os import path
from datetime import datetime

from IPython.core.debugger import set_trace



In [ ]:
class CollegeCrawl:
    gap_Insecond=5
    max_Pages=5
    headers = {'User-Agent': 'Mozilla/5.0'}



    def __init__(self,_collegename, _rooturl, 
            _prioritykeywords=['apply', 'admission', 'application', 'deadline'], 
           # _url_file=None,            
            _save_to_folder=None,
            _respectrobottxt=True):
    
        self.college=_collegename
         
        if urlparse(_rooturl).scheme=="":
            print('URL needs to have scheme. Please try again.')
            raise Exception('URL needs to have scheme')            
            return
        self.rootUrl=_rooturl
        
        self.priorityKeywords=_prioritykeywords  
        
        if _save_to_folder==None or path.isdir(_save_to_folder)==False:
            self.save_to_folder=os.getcwd()
        else:
            self.save_to_folder=_save_to_folder
        
        self.existingurlfile=path.join(self.save_to_folder,re.sub("\s+?", "_", self.college.strip()+'.csv'))
            
        self.respectRobottext=_respectrobottxt #currently not doing anything
        
        #self.allUrls=set()
        #self.allRankedUrls = [] 
        self.allurls={}
        self.rejectedUrls=set() 
        self.visitedUrls=set()
        self.scheme=urlparse(self.rootUrl).scheme
        self.base_domain=urlparse(self.rootUrl).netloc
        self.files=[]
    
    def __str__(self):
        return "************Crawl {}({})************".format(self.college, self.rootUrl)
    
        
    """
    def __getDomainFromUrl(self, url):
        #url = "http://www.mit.edu/"
        firstIndex = url.find('//')
        length = len(url)-1 if url.endswith('/') else len(url)      
        return url[firstIndex+2:length]
    """
    
    def __addAllUrlsInOnePage(self, response):
        soup=BeautifulSoup(response.text, 'html.parser') 
        for link in soup.find_all('a'): 
            if link.has_attr('href'):
                url=link['href']       
                url=re.sub(r'[\/_+!@#$?\\\s]+$', '', url)
                parsed_uri = urlparse(url) 
                absolute_url=''
           
                if (parsed_uri.netloc=='') and (parsed_uri.scheme=='') and re.match(r'^\/.*\w$', parsed_uri.path) :
                    absolute_url=urljoin(self.scheme+'://'+self.base_domain,parsed_uri.path)
                elif parsed_uri.netloc==self.base_domain and re.match('^http', parsed_uri.scheme):
                    absolute_url=url
                elif parsed_uri.netloc==self.base_domain and parsed_uri.scheme=="" and re.match(r'^\/.*\w$', parsed_uri.path):
                    absolute_url=self.scheme+'://'+parsed_uri.netloc+parsed_uri.path
                else:
                    continue 
                    
                if absolute_url!='' and absolute_url not in urls:   
                    urls[absolute_url]=0
        

    """
        get all urls starting from rootUrl
    """
    def getAllUrls(self):  
        urls=self.Load_DiscoveredUrls() 
        if len(urls)==0:
            urls={self.rootUrl:0}  
            unvisited=[self.rootUrl]
        else:
            unvisited=[url for url, status_code in urls.items() if status_code==0]
           # if not unvisited:
           #    unvisited=[self.rootUrl]
        
        pages_visited =0
        try:  
            while unvisited:        
                pages_visited+=1
                if pages_visited>self.max_Pages:
                    break 
                    
                unvisited=sorted(unvisited, key=lambda item: (sum([w in item for w in self.priorityKeywords])*100+10)/len(item))
                url=unvisited.pop()  
                response=requests.get(url, headers=headers)     
                status_code=response.status_code
                urls[url]=status_code
                
                # save all urls in the page to allurls dictionary with status code 0
                contentType = response.headers.get('content-type')    
                if status_code==200 and 'text/html' in contentType:
                    self.__addAllUrlsInOnePage(response)
                    
                # write the page to csv
                self.SaveToCsv_FromResponse(url, response)     
             
                unvisited=[name for name, code in urls.items() if code==0]          
                time.sleep(self.Gap_Insecond) #wait for few seconds. 
          
        except: 
            print('url "{}" went wrong'.format(url))  
            urls[url]=999
                #not to consider failed pages. status_code 400s may need manual handling of they are high priority pages
        finally: 
            self.allurls=urls
            #csv_columns = ['url', 'status_code']  
            #try:
                #with open(self.existingurlfile, 'w', newline='', encoding='utf-8') as csvfile:
                    #writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                    #writer.writeheader()
                    #for data in urls:
                        #writer.writerow(data)
            #except IOError:
                #print("I/O error")
           # self.Save_Summaries()   
        
    """
        Save One Page
        filename, if not None, should not be full name. use import ntpath ntpath.basename("a\b\c")
    """
    def SaveToCsv_FromUrl(self,url): # tab delimiter only 
        url=url.strip() 
        content=self.Read_Oneurl(url) #in format of (a,a,a,a)   
        self.SaveToCsv(url, content)
        #return SaveToCsv_FromResponse()
    
    '''
        save from resonse. called in the initial loop
    '''
    def SaveToCsv_FromResponse(self, url, response):
        content= self.Get_Pagetext(response)
        self.SaveToCsv(url, content) 
        
    '''
        save to csv file and append it to self.files
    '''    
    def SaveToCsv(self, url, content):
        filename=url.replace('.', '_dot_').replace('/', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
        fullname=path.join(self.save_to_folder, filename)
        try:
            with open(fullname, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
                writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
                for lll in content: 
                    lll.insert(0, url)
                    writer.writerow(lll)  
            self.files.append(fullname) 
        except IOError:
            print('failed to save file.')
           
        
    
    """
        read one page
    """
    def read_OneUrl(self, url):
        response=requests.get(url)  
        if response.status_code==200:
            if "text/html" in response.headers.get('content-type'):
                return self.get_pagetext(response)
            else: #for example .pdf file, img file
                pass #save the file directly?
        else: 
            return [[None, None, None, None]]
    
    
    def get_pagetext(self,body):
        soup = BeautifulSoup(body.text, 'html.parser') #another type is content which is byte (like image)
        texts = soup.findAll(text=True) 
        visible_texts = filter(self.tag_visible, texts)       
    
        return [ [t.parent.name,   
             t.parent.previousSibling.name if t.parent.previousSibling!=None else None, 
             t.nextSibling.name if t.nextSibling!=None else None,
             re.sub(r'[\s+\t]',' ',t) ]  for t in visible_texts if len(t.strip())>2] 
    
    def tag_visible(self,element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True 
        
    """
        Save One Page
    """
    def save_OnePage(self, url, format='csv'):
        url=url.strip()
        content=self.read_OneUrl(url) #in format of (a,a,a,a)
     
       # if folder==None:
        folder=os.getcwd()
        if path.isdir(folder)==False:
            print('folder doesnot exist')
            return
        #if filename==None:
        filename=url.replace('.', '_dot_').replace('/', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
    
        fullname=path.join(folder, filename)
        with open(fullname, 'w', newline='', encoding="utf-8") as newFile:
            writer = csv.writer(newFile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
            if format != 'csv':
               # writer = 
                pass
            writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
            for lll in content: 
                lll.insert(0, url)
                writer.writerow(lll)  
        #return fullname           
    
    """
        Save summaries
    """
    def save_Summaries(self):
        print(self)
        print("Total number of URLs visisted: ", len(self.visitedUrls))
        print("Total number of URLs rejected: ", len(self.rejectedUrls))

    """
        Ranks the urls based on the priority words
    """


    
    def setMaxpage(self, maxPages):
        CollegeCrawl.max_Pages = maxPages
        
    def setGap(self, gap):
        CollegeCrawl.gap_Insecond = gap
    
    
    def crawl(self):
        self.getAllUrls()
        self.save_Summaries()
        
     
            
        
        
    
    def Load_DiscoveredUrls(self, delimiter=',', hasHeader=False, header_names=['url', 'status_code']):   
        if self.existingurlfile==None:
            return {}
        else:
            if path.exists(self.existingurlfile) and re.sub('\s+', '_', self.college) in ntpath.basename(self.existingurlfile):
                df_urls=pd.read_csv(self.existingurlfile,  delimiter=delimiter).iloc[:, 0:2]   
                header_row=list(df_urls.columns) 
            
                if re.match('^http', header_row[0]):
                    df_urls.columns=header_names
                    df_urls=pd.concat([df_urls, pd.DataFrame([header_row], columns=header_names)])                
                else: #event it is not, still assign the header_name
                    df_urls.columns=header_names 
                
                return dict(zip(df_urls['url'], df_urls['status_code'])) #format: url:status_code. i.e., url is the key
                #another options is: df_urls.to_dict('list') #format of {url:[url1, url2...], status_code:[0, 200...]}
            else:
                return {}        

   

In [ ]:
colleges = [["UCLA", "http://www.ucla.edu", []],
           ["Stanford", "https://www.stanford.edu/", []],
           ["Yale", "https://www.yale.edu/", []],
           ["UW", "http://www.washington.edu/", []]]
for college in colleges:
    cr = CollegeCrawl(college[0], college[1], college[2])
    cr.crawl()


In [ ]:
def getResponse(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url)
    #response = requests.get(url,None, headers=headers)
    print("response status code for URL {} is {}".format(url,response.status_code))
    print("request headers for URL {} is {}".format(url, response.request.headers))
    
    
    #help(response)
    #help(requests.get)
    
    print(response.text)
 


getResponse("https://www.mit.edu")
#getResponseCode("http://www.stanford.edu")
#getResponseCode("http://www.harvard.edu")

In [ ]:
%debug

In [ ]:
_ih[-5:]